The following notebook has been forked from: https://www.kaggle.com/arvissu/simple-pytorch-lstm/comments?scriptVersionId=25732138 and only modified to respect the experimentation repeteability and metrics obtention.

In [1]:
target_columns = ['question_asker_intent_understanding', 'question_body_critical', 'question_conversational',
                  'question_expect_short_answer', 'question_fact_seeking', 'question_has_commonly_accepted_answer',
                  'question_interestingness_others', 'question_interestingness_self', 'question_multi_intent',
                  'question_not_really_a_question', 'question_opinion_seeking', 'question_type_choice',
                  'question_type_compare', 'question_type_consequence', 'question_type_definition',
                  'question_type_entity', 'question_type_instructions', 'question_type_procedure',
                  'question_type_reason_explanation', 'question_type_spelling', 'question_well_written',
                  'answer_helpful', 'answer_level_of_information', 'answer_plausible', 'answer_relevance',
                  'answer_satisfaction', 'answer_type_instructions', 'answer_type_procedure',
                  'answer_type_reason_explanation', 'answer_well_written']

In [2]:
import torch
import numpy as np
import random
import os

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

def set_seeds(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)

SEED = 21937
set_seeds(SEED)

SVD_QTY = 150

In [3]:
import gc
import re
import os
import sys
import time
import pickle
import random
import unidecode
import pandas as pd
from tqdm import tqdm
tqdm.pandas()
from scipy.stats import spearmanr
from gensim.models import Word2Vec
from flashtext import KeywordProcessor
from keras.preprocessing import text, sequence

import torch.nn as nn
import torch.nn.functional as F
from sklearn.model_selection import StratifiedKFold, GroupKFold


Using TensorFlow backend.


In [4]:
train = pd.read_csv('/kaggle/input/google-quest-challenge/train.csv')
test = pd.read_csv('/kaggle/input/google-quest-challenge/test.csv')
sub = pd.read_csv('/kaggle/input/google-quest-challenge/sample_submission.csv')

In [5]:
MODEL = pickle.load(open('/kaggle/input/pickled-glove840b300d-for-10sec-loading/glove.840B.300d.pkl', 'rb'))

In [6]:
PUNCTS = {
            '》', '〞', '¢', '‹', '╦', '║', '♪', 'Ø', '╩', '\\', '★', '＋', 'ï', '<', '?', '％', '+', '„', 'α', '*', '〰', '｟', '¹', '●', '〗', ']', '▾', '■', '〙', '↓', '´', '【', 'ᴵ',
            '"', '）', '｀', '│', '¤', '²', '‡', '¿', '–', '」', '╔', '〾', '%', '¾', '←', '〔', '＿', '’', '-', ':', '‧', '｛', 'β', '（', '─', 'à', 'â', '､', '•', '；', '☆', '／', 'π',
            'é', '╗', '＾', '▪', ',', '►', '/', '〚', '¶', '♦', '™', '}', '″', '＂', '『', '▬', '±', '«', '“', '÷', '×', '^', '!', '╣', '▲', '・', '░', '′', '〝', '‛', '√', ';', '】', '▼',
            '.', '~', '`', '。', 'ə', '］', '，', '{', '～', '！', '†', '‘', '﹏', '═', '｣', '〕', '〜', '＼', '▒', '＄', '♥', '〛', '≤', '∞', '_', '[', '＆', '→', '»', '－', '＝', '§', '⋅', 
            '▓', '&', 'Â', '＞', '〃', '|', '¦', '—', '╚', '〖', '―', '¸', '³', '®', '｠', '¨', '‟', '＊', '£', '#', 'Ã', "'", '▀', '·', '？', '、', '█', '”', '＃', '⊕', '=', '〟', '½', '』',
            '［', '$', ')', 'θ', '@', '›', '＠', '｝', '¬', '…', '¼', '：', '¥', '❤', '€', '−', '＜', '(', '〘', '▄', '＇', '>', '₤', '₹', '∅', 'è', '〿', '「', '©', '｢', '∙', '°', '｜', '¡', 
            '↑', 'º', '¯', '♫', '#'
          }


mispell_dict = {"aren't" : "are not", "can't" : "cannot", "couldn't" : "could not",
"couldnt" : "could not", "didn't" : "did not", "doesn't" : "does not",
"doesnt" : "does not", "don't" : "do not", "hadn't" : "had not", "hasn't" : "has not",
"haven't" : "have not", "havent" : "have not", "he'd" : "he would", "he'll" : "he will", "he's" : "he is", "i'd" : "I would",
"i'd" : "I had", "i'll" : "I will", "i'm" : "I am", "isn't" : "is not", "it's" : "it is",
"it'll":"it will", "i've" : "I have", "let's" : "let us", "mightn't" : "might not", "mustn't" : "must not", 
"shan't" : "shall not", "she'd" : "she would", "she'll" : "she will", "she's" : "she is", "shouldn't" : "should not", "shouldnt" : "should not",
"that's" : "that is", "thats" : "that is", "there's" : "there is", "theres" : "there is", "they'd" : "they would", "they'll" : "they will",
"they're" : "they are", "theyre":  "they are", "they've" : "they have", "we'd" : "we would", "we're" : "we are", "weren't" : "were not",
"we've" : "we have", "what'll" : "what will", "what're" : "what are", "what's" : "what is", "what've" : "what have", "where's" : "where is",
"who'd" : "who would", "who'll" : "who will", "who're" : "who are", "who's" : "who is", "who've" : "who have", "won't" : "will not", "wouldn't" : "would not", "you'd" : "you would",
"you'll" : "you will", "you're" : "you are", "you've" : "you have", "'re": " are", "wasn't": "was not", "we'll":" will", "didn't": "did not", "tryin'":"trying"}


def clean_punct(text):
  text = str(text)
  for punct in PUNCTS:
    text = text.replace(punct, ' {} '.format(punct))
  
  return text

def _get_mispell(mispell_dict):
    mispell_re = re.compile('(%s)' % '|'.join(mispell_dict.keys()))
    return mispell_dict, mispell_re


In [7]:
kp = KeywordProcessor(case_sensitive=True)

In [8]:
for k, v in mispell_dict.items():
    kp.add_keyword(k, v)

In [9]:
def preprocessing(text):
    text = kp.replace_keywords(text)
    text = clean_punct(text)
    text = re.sub(r'\n\r', ' ', text)
    text = re.sub(r'\s{2,}', ' ', text)
    
    return text.split()

In [10]:
train['clean_title'] = train['question_title'].apply(lambda x : preprocessing(x))
train['clean_body'] = train['question_body'].apply(lambda x : preprocessing(x))
train['clean_answer'] = train['answer'].apply(lambda x : preprocessing(x))

test['clean_title'] = test['question_title'].apply(lambda x : preprocessing(x))
test['clean_body'] = test['question_body'].apply(lambda x : preprocessing(x))
test['clean_answer'] = test['answer'].apply(lambda x : preprocessing(x))

In [11]:
y_columns = ['question_asker_intent_understanding',
       'question_body_critical', 'question_conversational',
       'question_expect_short_answer', 'question_fact_seeking',
       'question_has_commonly_accepted_answer',
       'question_interestingness_others', 'question_interestingness_self',
       'question_multi_intent', 'question_not_really_a_question',
       'question_opinion_seeking', 'question_type_choice',
       'question_type_compare', 'question_type_consequence',
       'question_type_definition', 'question_type_entity',
       'question_type_instructions', 'question_type_procedure',
       'question_type_reason_explanation', 'question_type_spelling',
       'question_well_written', 'answer_helpful',
       'answer_level_of_information', 'answer_plausible', 'answer_relevance',
       'answer_satisfaction', 'answer_type_instructions',
       'answer_type_procedure', 'answer_type_reason_explanation',
       'answer_well_written']

In [12]:
def build_matrix(word_index):

    embedding_matrix = np.zeros((len(word_index) + 1, 300))
    
    unknown_words = []
    unk = {}
    known_count = 0
    unk_count = 0
    for word, i in word_index.items():
        if word in MODEL:
            embedding_matrix[i] = MODEL[word]
            known_count += 1
            continue
        if word.lower() in MODEL:
            embedding_matrix[i] = MODEL[word.lower()]
            known_count += 1
            continue    
        if word.upper() in MODEL:
            embedding_matrix[i] = MODEL[word.upper()]
            known_count += 1
            continue
        if word.capitalize() in MODEL:
            embedding_matrix[i] = MODEL[word.capitalize()]
            known_count += 1
            continue
        if unidecode.unidecode(word) in MODEL:
            embedding_matrix[i] = MODEL[unidecode.unidecode(word)]
            known_count += 1
            continue
        try:
            unk[word] += 1 
        except:
            unk[word] = 1
        
        unk_count += 1
    
    
    print('all token in embedding percentage : {:.2f}%'.format( known_count/(unk_count+known_count)  * 100))
#     print('token in embedding percentage : {}'.format( known_count/(unk_count+known_count)))
    return embedding_matrix, unk

In [13]:
tokenizer = text.Tokenizer(filters='', lower=False)


tokenizer.fit_on_texts(list(train['clean_title']) + list(train['clean_body']) + list(train['clean_answer']) \
                        + list(test['clean_title']) + list(test['clean_body']) + list(test['clean_answer']))

In [14]:
TITLE_MAX_LEN = 50
BODY_MAX_LEN = 500
ANSWER_MAX_LEN = 500

In [15]:
train['clean_title_len'] = train['clean_title'].apply(lambda x : len(x))
train['clean_body_len'] = train['clean_body'].apply(lambda x : len(x))
train['clean_answer_len'] = train['clean_answer'].apply(lambda x : len(x))


test['clean_title_len'] = test['clean_title'].apply(lambda x : len(x))
test['clean_body_len'] = test['clean_body'].apply(lambda x : len(x))
test['clean_answer_len'] = test['clean_answer'].apply(lambda x : len(x))

# train title max 58 test 48

# train body max 4924 test body max 1894

# train answer max 8194 test max 2224

In [16]:
x_train_title = tokenizer.texts_to_sequences(train['clean_title'])
x_test_title = tokenizer.texts_to_sequences(test['clean_title'])

x_train_body = tokenizer.texts_to_sequences(train['clean_body'])
x_test_body = tokenizer.texts_to_sequences(test['clean_body'])

x_train_answer = tokenizer.texts_to_sequences(train['clean_answer'])
x_test_answer = tokenizer.texts_to_sequences(test['clean_answer'])


x_train_title = sequence.pad_sequences(x_train_title, maxlen=TITLE_MAX_LEN,padding='post')
x_test_title = sequence.pad_sequences(x_test_title, maxlen=TITLE_MAX_LEN,padding='post')

x_train_body = sequence.pad_sequences(x_train_body, maxlen=BODY_MAX_LEN,padding='post')
x_test_body = sequence.pad_sequences(x_test_body, maxlen=BODY_MAX_LEN,padding='post')

x_train_answer = sequence.pad_sequences(x_train_answer, maxlen=ANSWER_MAX_LEN,padding='post')
x_test_answer = sequence.pad_sequences(x_test_answer, maxlen=ANSWER_MAX_LEN,padding='post')


In [17]:
from sklearn.preprocessing import OneHotEncoder


c = 'host'
onehotencoder = OneHotEncoder(sparse=False, categories='auto').fit(np.concatenate((train[c].values.reshape(-1, 1).astype('str'), test[c].values.reshape(-1, 1).astype('str'))))
train_trans = onehotencoder.transform(train[c].values.reshape(-1, 1).astype('str'))
test_trans = onehotencoder.transform(test[c].values.reshape(-1, 1).astype('str'))
for i in range(train_trans.shape[1]):
    train['{}_{}'.format(c, i)] = train_trans[:, i]
    test['{}_{}'.format(c, i)] = test_trans[:, i]
print('remove origin column : {}'.format(c))
train = train.drop(columns=c)
test = test.drop(columns=c)
gc.collect()

remove origin column : host


0

In [18]:
train_category = pd.get_dummies(train['category'].values).values
test_category = pd.get_dummies(test['category'].values).values

hosts = ['host_{}'.format(i) for i in range(64)]
train_host = train.loc[:, hosts].values
test_host = test.loc[:, hosts].values

In [19]:
word2vec_matrix, unk = build_matrix(tokenizer.word_index)

all token in embedding percentage : 79.00%


In [20]:
class Attention(nn.Module):
    def __init__(self, feature_dim, step_dim, bias=True, **kwargs):
        super(Attention, self).__init__(**kwargs)

        self.supports_masking = True
        self.bias = bias
        self.feature_dim = feature_dim
        self.step_dim = step_dim
        self.features_dim = 0

        weight = torch.zeros(feature_dim, 1)
        nn.init.xavier_uniform_(weight)
        self.weight = nn.Parameter(weight)

        if bias:
            self.b = nn.Parameter(torch.zeros(1))

    def forward(self, x, mask=None):

        feature_dim = self.feature_dim
        step_dim = self.step_dim

        eij = torch.mm(
            x.contiguous().view(-1, feature_dim), 
            self.weight
        ).view(-1, step_dim)

        if self.bias:
            eij = eij + self.b

        eij = torch.tanh(eij)
        a = torch.exp(eij)

        if mask is not None:
            a = a * mask

        a = a / torch.sum(a, 1, keepdim=True) + 1e-10

        weighted_input = x * torch.unsqueeze(a, -1)
        return torch.sum(weighted_input, 1)

class SpatialDropout(nn.Module):
    def __init__(self,p):
        super(SpatialDropout, self).__init__()
        self.dropout = nn.Dropout2d(p)
        
    def forward(self, x):
        x = x.permute(0, 2, 1)   # convert to [batch, feature, timestep]
        x = self.dropout(x)
        x = x.permute(0, 2, 1)   # back to [batch, timestep, feature]
        return x

class LSTM_Model(nn.Module):
    def __init__(self, embedding_matrix, hidden_unit, num_layer=1):
        super(LSTM_Model, self).__init__()
        self.max_feature = embedding_matrix.shape[0]
        self.embedding_size = embedding_matrix.shape[1]
      
        self.embedding_body = nn.Embedding(self.max_feature, self.embedding_size)
        self.embedding_body.weight = nn.Parameter(torch.tensor(embedding_matrix, dtype=torch.float32))
        self.embedding_body.weight.required_grad = False
        
        self.embedding_answer = nn.Embedding(self.max_feature, self.embedding_size)
        self.embedding_answer.weight = nn.Parameter(torch.tensor(embedding_matrix, dtype=torch.float32))
        self.embedding_answer.weight.required_grad = False
        
        self.embedding_title = nn.Embedding(self.max_feature, self.embedding_size)
        self.embedding_title.weight = nn.Parameter(torch.tensor(embedding_matrix, dtype=torch.float32))
        self.embedding_title.weight.required_grad = False
        
        self.embedding_dropout = SpatialDropout(0.4)
        
        self.lstm1_body = nn.LSTM(self.embedding_size, hidden_unit, num_layers=num_layer, bidirectional=True, batch_first=True)
        self.lstm2_body = nn.LSTM(hidden_unit*2, int(hidden_unit/2), num_layers=num_layer, bidirectional=True, batch_first=True)
        
        self.lstm1_answer = nn.LSTM(self.embedding_size, hidden_unit, num_layers=num_layer, bidirectional=True, batch_first=True)
        self.lstm2_answer = nn.LSTM(hidden_unit*2, int(hidden_unit/2), num_layers=num_layer, bidirectional=True, batch_first=True)
        
        self.lstm1_title = nn.LSTM(self.embedding_size, hidden_unit, num_layers=num_layer, bidirectional=True, batch_first=True)
        self.lstm2_title = nn.LSTM(hidden_unit*2, int(hidden_unit/2), num_layers=num_layer, bidirectional=True, batch_first=True)
        
        self.attention_body = Attention(hidden_unit, BODY_MAX_LEN)
        self.attention_answer = Attention(hidden_unit, ANSWER_MAX_LEN)
        self.attention_title = Attention(hidden_unit, TITLE_MAX_LEN)
        
#         self.category = nn.Embedding(5, 10)
#         self.host = nn.Embedding(64, 128)
        
        self.linear_title = nn.Linear(hidden_unit*3, hidden_unit)
        self.linear_body = nn.Linear(hidden_unit*3, hidden_unit)
        self.linear_answer = nn.Linear(hidden_unit*3, hidden_unit)
        
#         self.linear_out = nn.Linear(hidden_unit, 30)
        self.additional_category = nn.Linear(5, 5)
        self.additional_host = nn.Linear(64, 32)
        
        self.linear_q = nn.Linear(hidden_unit*2+37, hidden_unit)
        self.linear_a = nn.Linear(hidden_unit+37, hidden_unit)
        self.linear_q_out = nn.Linear(hidden_unit, 21)
        self.linear_a_out = nn.Linear(hidden_unit, 9)
        
    def forward(self, body, answer, title, category, host):
        
        x_body = self.embedding_dropout(self.embedding_body(body))
        h_lstm1_body, _ = self.lstm1_body(x_body)
        h_lstm2_body, _ = self.lstm2_body(h_lstm1_body)
        
        x_answer = self.embedding_dropout(self.embedding_answer(answer))
        h_lstm1_answer, _ = self.lstm1_answer(x_answer)
        h_lstm2_answer, _ = self.lstm2_answer(h_lstm1_answer)
        
        x_title = self.embedding_dropout(self.embedding_title(title))
        h_lstm1_title, _ = self.lstm1_title(x_title)
        h_lstm2_title, _ = self.lstm2_title(h_lstm1_title)
        
#         print(h_lstm2_body.size())
        att_body = self.attention_body(h_lstm2_body)
        att_answer = self.attention_answer(h_lstm2_answer)
        att_title = self.attention_title(h_lstm2_title)
        
        avg_pool_body = torch.mean(h_lstm2_body, 1)
        max_pool_body, _ = torch.max(h_lstm2_body, 1)
        
        avg_pool_answer = torch.mean(h_lstm2_answer, 1)
        max_pool_answer, _ = torch.max(h_lstm2_answer, 1)
        
        avg_pool_title = torch.mean(h_lstm2_title, 1)
        max_pool_title, _ = torch.max(h_lstm2_title, 1)
        
        body_cat = torch.cat((att_body, avg_pool_body, max_pool_body), 1)
        answer_cat = torch.cat((att_answer, avg_pool_answer, max_pool_answer), 1)
        title_cat = torch.cat((att_title, avg_pool_title, max_pool_title), 1)
        
        body_cat = torch.relu(self.linear_body(body_cat))
        answer_cat = torch.relu(self.linear_answer(answer_cat))
        title_cat = torch.relu(self.linear_title(title_cat))

        category = self.additional_category(category)
        host = self.additional_host(host)
        
        hidden_q = self.linear_q(torch.cat((title_cat, body_cat, category, host), 1))
        hidden_a = self.linear_a(torch.cat((answer_cat, category, host), 1))
                                          
        q_result = self.linear_q_out(hidden_q)
        a_result = self.linear_a_out(hidden_a)
        
        out = torch.cat([q_result, a_result], 1)
        return out

In [21]:
SEED = 2020
BATCH_SIZE = 32
EPOCHS = 20
LR = 0.001
hidden_unit = 256

In [22]:
NFOLDS = 5
scores = []

kf = GroupKFold(n_splits=NFOLDS).split(X=train.question_body, groups=train.question_body)

In [23]:
class TextDataset(torch.utils.data.TensorDataset):

    def __init__(self, body_data, answer_data, title_data, category_data, host_data, idxs, targets=None):
        self.body_data = body_data[idxs]
        self.answer_data = answer_data[idxs]
        self.title_data = title_data[idxs]
        self.category_data = category_data[idxs]
        self.host_data = host_data[idxs]
        self.targets = targets[idxs] if targets is not None else np.zeros((self.body_data.shape[0], 30))

    def __getitem__(self, idx):
        body = self.body_data[idx]
        answer = self.answer_data[idx]
        title = self.title_data[idx]
        category = self.category_data[idx]
        host = self.host_data[idx]
        target = self.targets[idx]

        return body, answer, title, category, host, target

    def __len__(self):
        return len(self.body_data)

In [24]:
test_loader = torch.utils.data.DataLoader(TextDataset(x_test_body, x_test_answer, x_test_title, test_category, test_host, test.index),
                          batch_size=BATCH_SIZE, shuffle=False)

In [25]:
gc.collect()

0

In [26]:
def compute_spearmanr(preds, trues):
    rhos = []
    for col_trues, col_pred in zip(trues.T, preds.T):
        rhos.append(
            spearmanr(col_trues, col_pred + np.random.normal(0, 1e-7, col_pred.shape[0])).correlation)
    return np.nanmean(rhos), rhos


In [27]:
y = train.loc[:, y_columns].values

oof = np.zeros((len(train), 30))
test_pred = np.zeros((len(test), 30))


for i, (train_idx, valid_idx) in enumerate(kf):
    print(f'fold {i+1}')
    gc.collect()
    train_loader = torch.utils.data.DataLoader(TextDataset(x_train_body, x_train_answer, x_train_title, train_category, train_host, train_idx, y),
                          batch_size=BATCH_SIZE, shuffle=True, pin_memory=True)

    val_loader = torch.utils.data.DataLoader(TextDataset(x_train_body, x_train_answer, x_train_title, train_category, train_host, valid_idx, y),
                          batch_size=BATCH_SIZE, shuffle=False, pin_memory=True)

    net = LSTM_Model(word2vec_matrix, hidden_unit)
    net.cuda()
    loss_fn = torch.nn.BCEWithLogitsLoss(reduction='mean').cuda()
    optimizer = torch.optim.Adam(net.parameters(), lr=LR)
    for epoch in range(EPOCHS):  
        start_time = time.time()
        avg_loss = 0.0
        net.train()
        for data in train_loader:

            # get the inputs
            body, answer, title, category, host, labels = data
            pred = net(body.long().cuda(), answer.long().cuda(), title.long().cuda(), category.float().cuda(), host.float().cuda())

            loss = loss_fn(pred, labels.cuda())
            # Before the backward pass, use the optimizer object to zero all of the
            # gradients for the Tensors it will update (which are the learnable weights
            # of the model)
            optimizer.zero_grad()

            # Backward pass: compute gradient of the loss with respect to model parameters
            loss.backward()
            # Calling the step function on an Optimizer makes an update to its parameters
            optimizer.step()

            avg_loss += loss.item()

        avg_val_loss = 0.0
        net.eval()

        valid_preds = np.zeros((len(valid_idx), 30))
        true_label = np.zeros((len(valid_idx), 30))
        for j, data in enumerate(val_loader):

            # get the inputs
            body, answer, title, category, host, labels = data

            ## forward + backward + optimize
            pred = net(body.long().cuda(), answer.long().cuda(), title.long().cuda(), category.float().cuda(), host.float().cuda())

            loss_val = loss_fn(pred, labels.cuda())
            avg_val_loss += loss_val.item()

            valid_preds[j * BATCH_SIZE:(j+1) * BATCH_SIZE] = torch.sigmoid(pred).cpu().detach().numpy()
            true_label[j * BATCH_SIZE:(j+1) * BATCH_SIZE]  = labels

        score = 0

        score, rho_cols = compute_spearmanr(valid_preds, true_label)

        oof[valid_idx] = valid_preds
        elapsed_time = time.time() - start_time 
        current_loss = avg_loss / len(train_loader)
        print(f'epoch = {epoch}, train_loss = {current_loss}, valid_spearman = {score}')

    rho_print = [print(target_columns[i] + " rho: " + str(rho_cols[i]) ) for i in range(0, len(target_columns))]      


    test_pred_fold = np.zeros((len(test), 30))

    with torch.no_grad():
        for q, data in enumerate(test_loader):
            body, answer, title, category, host, _ = data
            y_pred = net(body.long().cuda(), answer.long().cuda(), title.long().cuda(), category.float().cuda(), host.float().cuda())
            test_pred_fold[q * BATCH_SIZE:(q+1) * BATCH_SIZE] = torch.sigmoid(y_pred).cpu().detach().numpy()
    test_pred += test_pred_fold/NFOLDS        


fold 1
epoch = 0, train_loss = 0.4190459871994475, valid_spearman = 0.30170456079546326
epoch = 1, train_loss = 0.39031264487073447, valid_spearman = 0.33223485107031686
epoch = 2, train_loss = 0.3764026749650016, valid_spearman = 0.3399905226473844
epoch = 3, train_loss = 0.3663959198019981, valid_spearman = 0.34428076611016484
epoch = 4, train_loss = 0.3554971591998881, valid_spearman = 0.3453158287716578
epoch = 5, train_loss = 0.34737179120392264, valid_spearman = 0.34511765639272396
epoch = 6, train_loss = 0.34064099666133457, valid_spearman = 0.3411208041914977
epoch = 7, train_loss = 0.333929943524048, valid_spearman = 0.33045270397886733
epoch = 8, train_loss = 0.32846116309885476, valid_spearman = 0.3285881387410559
epoch = 9, train_loss = 0.3232652291763161, valid_spearman = 0.3275833805566668
epoch = 10, train_loss = 0.3196297278818287, valid_spearman = 0.3298948776335171
epoch = 11, train_loss = 0.3159830068324626, valid_spearman = 0.3225660700270118
epoch = 12, train_loss 

In [28]:
sub.loc[:, y_columns] = test_pred
sub.to_csv('submission.csv', index=False)

In [29]:
sub.head()

,qa_id,question_asker_intent_understanding,question_body_critical,question_conversational,question_expect_short_answer,question_fact_seeking,question_has_commonly_accepted_answer,question_interestingness_others,question_interestingness_self,question_multi_intent,...,question_well_written,answer_helpful,answer_level_of_information,answer_plausible,answer_relevance,answer_satisfaction,answer_type_instructions,answer_type_procedure,answer_type_reason_explanation,answer_well_written
0,39,0.918506,0.645757,0.064284,0.573711,0.884746,0.807765,0.655760,0.560490,0.677703,...,0.924474,0.987221,0.629140,0.997338,0.995638,0.898412,0.008936,0.004770,0.981833,0.924706
1,46,0.811063,0.489197,0.005883,0.792464,0.701580,0.968416,0.509932,0.383948,0.007589,...,0.587967,0.950075,0.631658,0.983467,0.985830,0.863464,0.993319,0.063081,0.002591,0.872336
2,70,0.865339,0.596926,0.040526,0.650133,0.921944,0.887278,0.580020,0.472936,0.094671,...,0.856532,0.960352,0.621333,0.984682,0.984568,0.858411,0.056000,0.065011,0.762475,0.901413
3,132,0.886929,0.471186,0.001069,0.830438,0.858176,0.976739,0.498886,0.411981,0.041162,...,0.758164,0.983896,0.717965,0.992478,0.993596,0.928597,0.647577,0.338250,0.623806,0.933665
4,200,0.958579,0.430061,0.008225,0.969150,0.899428,0.935114,0.687724,0.711467,0.175309,...,0.817338,0.902702,0.613196,0.950377,0.945369,0.786259,0.209996,0.483130,0.403464,0.884306
